# Przetwarzanie języka naturalnego – lab8
## Mateusz Kocot

In [1]:
import os
import regex as re
import time
import json
from pprint import pprint

import pandas as pd

from haystack.document_stores import ElasticsearchDocumentStore, FAISSDocumentStore
from haystack.nodes import DensePassageRetriever
from haystack.nodes import ElasticsearchRetriever

from IPython.display import display, HTML

C:\Users\MatiX\anaconda3\envs\nlp-py39\lib\site-packages\ray\autoscaler\_private\cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [2]:
DATA_DIR = '../ustawy'

# Task 3

In [3]:
elastic_store = ElasticsearchDocumentStore()
elastic_retriever = ElasticsearchRetriever(elastic_store)

Creating the store and retriever for the first time

In [4]:
# faiss_store = FAISSDocumentStore(faiss_index_factory_str="Flat")
# faiss_retriever = DensePassageRetriever(
#     document_store=faiss_store,
#     query_embedding_model="enelpol/czywiesz-question",
#     passage_embedding_model="enelpol/czywiesz-context",
# )

Loading the store and retriever with saved and processed documents

In [5]:
faiss_store = FAISSDocumentStore.load(index_path='faiss/documents.faiss')
faiss_retriever = DensePassageRetriever.load('faiss/retriever.pt', document_store=faiss_store)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at faiss\retriever.pt\query_encoder
INFO - haystack.modeling.model.language_model -  Loaded faiss\retriever.pt\query_encoder
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at faiss\retriever.pt\passage_encoder
INFO - haystack.modeling.model.language_model -  Loaded faiss\retriever.pt\passage_encoder
INFO - haystack.nodes.retriever.dense -  DPR model loaded from faiss\retriever.pt


# Task 4

In [6]:
original_dataset = {}

for file_name in os.listdir(DATA_DIR):
    with open(f'{DATA_DIR}/{file_name}', 'r', encoding='utf-8') as file:
        bill = file.read()
    original_dataset[file_name[:-4]] = bill

In [7]:
def produce_passages(text):
    passages = {}
    matches = list(re.finditer(r'\s*Art.\s*(\d+).?\s*', text))
    for i, m in enumerate(matches):
        art_number = m.group(1)
        art_start = m.span()[1]
        art_end = matches[i + 1].span()[0] if i + 1 < len(matches) else len(text)
        art_text = text[art_start:art_end]
        passages[art_number] = art_text
    return passages

articles = []
for act_id, act_text in original_dataset.items():
    for art_number, content in produce_passages(act_text).items():
        articles.append({
            'content': content,
            'meta': {
                'act_id': act_id,
                'art_number': art_number
            }
        })

# Task 5
- Elastic store: 64 s
- Faiss store: 48 s (writing documents) + 226 s (updating embeddings) = 274 s

In [8]:
# start = time.time()
# elastic_store.write_documents(articles)
# print(f'Elapsed {time.time() - start:0.2f} s')

In [9]:
# start = time.time()
# faiss_store.write_documents(articles)
# print(f'Writing. Elapsed {time.time() - start:0.2f} s')

# start = time.time()
# faiss_store.update_embeddings(faiss_retriever)
# print(f'Updating embeddings. Elapsed {time.time() - start:0.2f} s')

# faiss_store.save('faiss/documents.faiss')
# faiss_retriever.save('faiss/retriever.pt')

# Task 6

In [10]:
questions = {}
with open('questions.jl', encoding='utf-8') as file:
    for line in file:
        data = json.loads(line)
        q_id = int(data['_id'])
        questions[q_id] = {}
        questions[q_id]['question'] = data['text']
        
with open('context_mappings.jl', encoding='utf-8') as file:
    for line in file:
        data = json.loads(line)
        q_id = int(data['question-id'])
        m = re.match(r'([^_]*_[^_]*)_(.*)', data['passage-id'])
        questions[q_id]['act_id'] = m.group(1)
        questions[q_id]['art_number'] = m.group(2)
        
pprint(questions, width=160)

{1345: {'act_id': '1997_604', 'art_number': '39', 'question': 'Czy źródła finansowania partii politycznych mogą być niejawne?'},
 1346: {'act_id': '1998_1118', 'art_number': '69', 'question': 'Czy wnuki przyjęte na wychowanie mają prawo do renty po zmarłych rodzicach?'},
 1347: {'act_id': '1997_769', 'art_number': '30', 'question': 'Kiedy kurator składa wniosek o ogłoszenie upadłości osoby prawnej?'},
 1348: {'act_id': '1996_110',
        'art_number': '223',
        'question': 'Czy dawka promieniowania jonizującego pochodzącego ze źródeł naturalnych może przekroczyć dawkę graniczną?'},
 1349: {'act_id': '2001_18',
        'art_number': '14',
        'question': 'Czy dawka promieniowania jonizującego pochodzącego ze źródeł naturalnych może przekroczyć dawkę graniczną?'},
 1350: {'act_id': '1996_463', 'art_number': '1', 'question': 'Jaki tytuł ma obecnie Wyższa Szkoła Inżynierska w Koszalinie?'},
 1351: {'act_id': '2001_1207',
        'art_number': '20',
        'question': 'Czy wydani

In [11]:
# Check if the expected passages are available in the dataset
available_passages = [(a['meta']['act_id'], a['meta']['art_number']) for a in articles]
for q in questions.values():
    assert (q['act_id'], q['art_number']) in available_passages

# Task 6

In [12]:
for q_id, q in list(questions.items()):
    display(HTML(f"<h2>{q_id}: {q['question']} ({q['act_id']}, {q['art_number']})</h2>"))
    for retriever, retriever_name in [(elastic_retriever, 'Elastic'), (faiss_retriever, 'Faiss')]:
        display(HTML(f"<h4>{retriever_name} retriever</h4>"))
        
        possible_passages = retriever.retrieve(query=q['question'], top_k=1)
        for p in possible_passages:
            print(f"Document {p.meta['act_id']}, Art. {p.meta['art_number']}:")
            print(p.content)
            print('---------------------------------------------------------------------------------------------')

Document 2001_499, Art. 23:
. Źródła finansowania partii politycznych są jawne.";
4) art. 24 i 25 otrzymują brzmienie:
      "
---------------------------------------------------------------------------------------------


Document 1997_604, Art. 41:
Partie polityczne prowadzą rachunkowość na zasadach określonych odrębnymi 
przepisami z uwzględnieniem przepisów niniejszej ustawy.
                               Rozdział 5
     Postępowanie w sprawie stwierdzenia sprzeczności z Konstytucją
               celów lub działalności partii politycznych
---------------------------------------------------------------------------------------------


Document 1998_1118, Art. 69:
.
Przyjęte na wychowanie i utrzymanie wnuki, rodzeństwo i inne dzieci, o których
mowa w art. 67 ust. 1 pkt 2, mają prawo do renty rodzinnej, jeżeli spełniają
warunki określone w art. 68, a ponadto:
   1) zostały przyjęte na wychowanie i utrzymanie co najmniej na rok przed
     śmiercią ubezpieczonego (emeryta lub rencisty), chyba że śmierć była
     następstwem wypadku, oraz
   2) nie mają prawa do renty po zmarłych rodzicach, a gdy rodzice żyją,
     jeżeli:
     a) nie mogą zapewnić im utrzymania albo
     b) ubezpieczony (emeryt lub rencista) lub jego małżonek był ich
       opiekunem ustanowionym przez sąd.
---------------------------------------------------------------------------------------------


Document 1998_1118, Art. 71:
Rodzice mają prawo do renty rodzinnej, jeżeli:
   1) ubezpieczony (emeryt lub rencista) bezpośrednio przed śmiercią
     przyczyniał się do ich utrzymania,
   2) spełniają odpowiednio warunki określone dla wdowy i wdowca w art. 70
     ust. 1 i 2 oraz, co do wieku, również w art. 70 ust. 5.
---------------------------------------------------------------------------------------------


Document 1997_769, Art. 30:
.
Kurator składa wniosek o ogłoszenie upadłości osoby prawnej, dla której został
ustanowiony, jeżeli stwierdzi, że istnieją podstawy do ogłoszenia upadłości.
---------------------------------------------------------------------------------------------


...

Document 2001_627, Art. 1:
Ustawa określa zasady ochrony środowiska oraz warunki korzystania z jego
zasobów, z uwzględnieniem wymagań zrównoważonego rozwoju, a w szczególności:
   1) zasady ustalania:
     a) warunków ochrony zasobów środowiska,
     b) warunków wprowadzania substancji lub energii do środowiska,
     c) kosztów korzystania ze środowiska,
   2) udostępnianie informacji o środowisku i jego ochronie,
   3) udział społeczeństwa w postępowaniu w sprawie ochrony środowiska,
   4) obowiązki organów administracji,
   5) odpowiedzialność i sankcje.
---------------------------------------------------------------------------------------------


Document 1997_468, Art. 29:
.
 1. Preparaty zawierające środki odurzające grup I-N, II-N i III-N lub substancje
   psychotropowe grup II-P, III-P i IV-P, które zostały dopuszczone 
   do obrotu jako środki farmaceutyczne na podstawie przepisów, o których 
   mowa w art. 2 pkt 1, może posiadać, w celach medycznych, zakład opieki 
   zdrowotnej nie mający apteki zakładowej, zakład leczniczy dla zwierząt 
   oraz lekarz, lekarz stomatolog lub lekarz weterynarii, prowadzący indywidualną
   praktykę lekarską, oraz inne podmioty.
 2. Minister Zdrowia i Opieki Społecznej określi, w drodze rozporządzenia, 
   rodzaje preparatów i ich ilości, jakie mogą posiadać podmioty, o których 
   mowa w ust. 1 oraz inne podmioty, szczegółowe warunki przechowywania 
   tych preparatów oraz prowadzenia dokumentacji w zakresie ich posiadania
   i stosowania.
---------------------------------------------------------------------------------------------


Document 1997_153, Art. 42:
Minister Zdrowia i Opieki Społecznej po porozumieniu z Radą Krajowego 
Związku Kas oraz po zasięgnięciu opinii Naczelnej Rady Lekarskiej i Naczelnej
Rady Aptekarskiej określa, w drodze rozporządzenia, leki i artykuły medyczne,
które nie mogą być przepisywane na recepcie uprawniającej do nabycia 
leku lub artykułu sanitarnego na zasadach określonych w art. 40 i 41.
---------------------------------------------------------------------------------------------


Document 2004_880, Art. 94:
.
Zadania wojewody w zakresie ochrony przyrody na terenie parku narodowego
wykonuje dyrektor tego parku.
---------------------------------------------------------------------------------------------


Document 2004_880, Art. 94:
Zadania wojewody w zakresie ochrony przyrody na terenie parku narodowego
wykonuje dyrektor tego parku.
---------------------------------------------------------------------------------------------


# Task 7
- Calculating Pr@3 doesn't make a lot of sense, since none of the questions in my dataset have three correct answers... However, we can still use it compare the two methods.
- I'm using the exact formulas from the assignment, so, e.g. if among 3 retrieved documents there are 2 valid documents (one document twice) out of 1, the Rc@3 is getting bigger than 1.0...

In [13]:
# retrieved from questions.jl and relevant.jl. If the question is present more than once, 
# I assume that it has more than one correct answer. 
ground_truth = {
    1345: [('2001_499', '238'), ('1997_604', '39')],
    1346: [('1998_1118', '69')],
    1347: [('1997_769', '30')],
    1348: [('1996_110', '223'), ('2001_18', '14')],
    1349: [('1996_110', '223'), ('2001_18', '14')],
    1350: [('1996_463', '1')],
    1351: [('2001_1207', '20')],
    1352: [('1997_593', '23')],
    1353: [('1997_553', '184')],
    1354: [('2001_1409', '27')],
    1355: [('2001_499', '18')],
    1356: [('1997_739', '86'), ('1997_739', '109')],
    1357: [('1997_739', '86'), ('1997_739', '109')],
    1358: [('2001_298', '126')],
    1359: [('1997_251', '6')],
    1360: [('1997_926', '232')],
    1361: [('2002_1802', '3')],
    1362: [('2001_18', '42')],
    1363: [('2001_627', '154')],
    1364: [('1997_468', '29')],
    1365: [('2004_880', '94')],
}

In [14]:
def calculate_stats(question_id, question, retriever, at=1):
    start = time.time()
    possible_passages = retriever.retrieve(query=question['question'], top_k=at)
    elapsed_time = time.time() - start
    possible_articles = [(p.meta['act_id'], p.meta['art_number']) for p in possible_passages]
    
    n_valid_answers = sum([a in ground_truth[question_id] for a in possible_articles])
    n_all_correct = len(ground_truth[question_id])
    
    pr = n_valid_answers / at
    rc = n_valid_answers / n_all_correct
    return pr, rc, elapsed_time

In [15]:
stats = {}
for q_id, q in list(questions.items()):
    stats[q_id] = []
    for retriever, retriever_name in [(elastic_retriever, 'Elastic'), (faiss_retriever, 'Faiss')]:
        pr1, rc1, t1 = calculate_stats(q_id, q, retriever, 1)
        pr3, rc3, t3 = calculate_stats(q_id, q, retriever, 3)
        stats[q_id] += [round(a, 2) for a in [pr1, rc1, pr3, rc3]] + [round(t1 + t3, 3)]

In [16]:
df = pd.DataFrame.from_dict(stats, orient='index', columns=['Pr@1 Elastic', 'Rc@1 Elastic', 'Pr@3 Elastic', 'Rc@3 Elastic', 'time Elastic', 'Pr@1 Faiss', 'Rc@1 Faiss', 'Pr@3 Faiss', 'Rc@3 Faiss', 'time Faiss'])
df = df.iloc[:, [0, 5, 1, 6, 2, 7, 3, 8, 4, 9]]
df

,Pr@1 Elastic,Pr@1 Faiss,Rc@1 Elastic,Rc@1 Faiss,Pr@3 Elastic,Pr@3 Faiss,Rc@3 Elastic,Rc@3 Faiss,time Elastic,time Faiss
1345,0.0,0.0,0.0,0.0,0.33,0.00,0.5,0.0,0.010,0.066
1346,1.0,0.0,1.0,0.0,0.67,0.33,2.0,1.0,0.008,0.066
1347,1.0,1.0,1.0,1.0,0.67,0.33,2.0,1.0,0.007,0.075
1348,1.0,0.0,0.5,0.0,0.67,0.33,1.0,0.5,0.007,0.069
1349,1.0,0.0,0.5,0.0,0.67,0.33,1.0,0.5,0.007,0.065
1350,0.0,1.0,0.0,1.0,0.00,0.33,0.0,1.0,0.006,0.065
1351,1.0,0.0,1.0,0.0,0.67,0.00,2.0,0.0,0.008,0.065
1352,1.0,1.0,1.0,1.0,0.67,0.33,2.0,1.0,0.005,0.065
1353,1.0,0.0,1.0,0.0,0.67,0.00,2.0,0.0,0.007,0.065
1354,1.0,1.0,1.0,1.0,0.67,0.33,2.0,1.0,0.007,0.065


# Task 9
#### Which of the document stores performs better? Take into account the different metrics enumerated in the previous point.
ElasticSearch document store is better considering the metrics.

#### Which of the document stores is faster?
Elastic is faster in processing documents and answering questions.

#### Try to determine the other pros and cons of using sparse and dense document retrieval models.
- Elastic (sparse) is often returning two same answers (that's why its Rc@3 is often 2.0)
- The advantage of dense models is the fact that they can be easily adjusted for a specific dataset.